In [1]:
import othello as oth
import cProfile
import agents as ag
import random as rd
import time
from collections import defaultdict
import mnk
import numpy as np
#import carcassonne as carc

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
base_gs = mnk.GameState(13,13,5)
base_gs.set_initial_state()
start = time.time()
games = 0
agent = ag.RandomPlayer()
wins = defaultdict(lambda: 0)

def random_games(games, state):
    wins = defaultdict(lambda:0)
    for _ in range(games):
        gs = state.duplicate()
        while not gs.is_terminal:
            action = agent.choose_action(gs)
            gs.make_action(action)
        wins[gs.winner] += 1
    return wins

def random_game(state):
    gs = state.duplicate()
    while not gs.is_terminal:
        action = agent.choose_action(gs)
        gs.make_action(action)
    return gs


In [13]:
gs = random_game(base_gs)
gs.view_game_state()

[['X' 'O' 'O' 'X' 'O' ' ' ' ' 'O' ' ' ' ' 'X' 'O' ' ']
 ['X' 'O' 'O' 'O' 'X' 'O' ' ' ' ' 'O' ' ' ' ' ' ' ' ']
 ['X' 'O' 'X' ' ' 'X' 'X' ' ' ' ' 'X' ' ' ' ' ' ' 'X']
 [' ' 'O' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'X' ' ' ' ']
 [' ' 'O' ' ' ' ' 'O' ' ' 'O' 'O' 'X' ' ' 'X' 'X' 'O']
 [' ' ' ' ' ' 'O' ' ' 'X' 'O' ' ' 'O' ' ' 'O' 'X' ' ']
 ['X' 'O' ' ' ' ' 'X' 'O' ' ' 'X' ' ' ' ' 'X' ' ' ' ']
 ['X' 'O' 'O' ' ' 'X' ' ' ' ' 'O' ' ' ' ' ' ' 'X' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'X' ' ']
 ['X' ' ' ' ' ' ' 'X' ' ' 'O' ' ' ' ' ' ' 'X' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'X' 'O' ' ' 'O' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'X' 'O' 'X' ' ' ' ' 'X' 'X']
 ['X' ' ' 'X' 'X' ' ' ' ' ' ' 'O' ' ' 'X' 'O' 'O' 'O']]


In [4]:
cProfile.run("wins =  random_games(1000, base_gs)")

         10016751 function calls (9833647 primitive calls) in 9.939 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    91552    0.064    0.000    0.520    0.000 <__array_function__ internals>:2(flipud)
        1    0.131    0.131    9.966    9.966 <ipython-input-2-b4a1b3ad2937>:8(random_games)
        2    0.000    0.000    0.000    0.000 <ipython-input-2-b4a1b3ad2937>:9(<lambda>)
        1    0.000    0.000    9.966    9.966 <string>:1(<module>)
    91552    0.034    0.000    0.108    0.000 _asarray.py:110(asanyarray)
    91552    0.051    0.000    0.227    0.000 agents.py:74(choose_action)
     1000    0.007    0.000    0.040    0.000 defmatrix.py:115(__new__)
824968/641864    0.482    0.000    0.558    0.000 defmatrix.py:167(__array_finalize__)
   274656    0.652    0.000    1.031    0.000 defmatrix.py:189(__getitem__)
   169000    0.035    0.000    0.035    0.000 mnk.py:10(__init__)
   169000    0.060    0.000    0.095

In [5]:
wins =  random_games(1000, base_gs)
print(wins)

defaultdict(<function random_games.<locals>.<lambda> at 0x000001AF1C4F5820>, {0: 491, 1: 509})


In [4]:
base_gs = mnk.GameState(5,5,3)
base_gs.set_initial_state()
agent = ag.RandomPlayer()
gs = base_gs.duplicate()

In [8]:

action = agent.choose_action(gs)
gs.make_action(action)
horizontal = gs.board[action.x]
vertical = gs.board[:,action.y]
offset = action.y-action.x
antioffset = gs.n - 1 - action.y - action.x
diagonal_up = gs.board.diagonal(offset)
diagonal_down = np.fliplr(gs.board).diagonal(antioffset)


gs.view_game_state()
print("horizontal", horizontal)
print("vertical", vertical.transpose())
print("action.x", action.x)
print("action.y", action.y)
print("offset", offset)
print("diagonal_up", diagonal_up)
print("diagonal_down", diagonal_down)

[[' ' ' ' ' ' ' ' ' ']
 ['O' 'X' ' ' ' ' ' ']
 [' ' ' ' 'X' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' 'O' ' ' ' ' ' ']]
horizontal [[2 1 2 2 2]]
vertical [[2 0 2 2 1]]
action.x 4
action.y 1
offset -3
diagonal_up [[2 1]]
diagonal_down [[2 2 2 1]]
